In [1]:
from keras import applications
from keras import models
from keras import layers
from keras import metrics
from keras.preprocessing import image

import matplotlib.pyplot as plt

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Using TensorFlow backend.


In [2]:
N_CLASSES = 2
BATCH_SIZE = 16
DROP_OUT = 0.3

In [3]:
## carregando o modelo pre-treinado

resnet50 = applications.ResNet50V2(include_top=False, weights = 'imagenet', input_shape = (224,224,3), pooling = None)

In [4]:
## congelando os layers
    
for layer in resnet50.layers:
    layer.trainable = False

## ResNet50

In [5]:
model_resnet50 = models.Sequential()
model_resnet50.add(resnet50)

model_resnet50.add(layers.Flatten(name='flattened'))
model_resnet50.add(layers.Dense(2048,activation='relu'))
model_resnet50.add(layers.Dropout(DROP_OUT, name='dropout'))
model_resnet50.add(layers.Dense(N_CLASSES, activation='softmax', name='predictions'))

model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.AUC(),'accuracy'])

model_resnet50.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
flattened (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              205522944 
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
predictions (Dense)          (None, 2)                 4098      
Total params: 229,091,842
Trainable params: 205,527,042
Non-trainable params: 23,564,800
_________________________________________________________________


## Dados

In [6]:
train_datagen = image.ImageDataGenerator(rescale = 1.0/255,
                                        data_format = 'channels_last',
                                        rotation_range = 30,
                                        horizontal_flip = True,
                                        fill_mode = 'reflect')

test_datagen = image.ImageDataGenerator(rescale = 1.0/255,
                                        data_format = 'channels_last')

In [7]:
train_generator = train_datagen.flow_from_directory('../0data/train/',
                                                   target_size = (224,224),
                                                   classes = ['NORMAL','PNEUMONIA'],
                                                   class_mode = 'categorical',
                                                   batch_size = BATCH_SIZE,
                                                   shuffle = True,
                                                   seed = 42)

test_generator = test_datagen.flow_from_directory('../0data/test/',
                                                   target_size = (224,224),
                                                   classes = ['NORMAL','PNEUMONIA'],
                                                   class_mode = 'categorical',
                                                   batch_size = BATCH_SIZE,
                                                   shuffle = True,
                                                   seed = 42)

Found 5220 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [8]:
resnet50 = model_resnet50.fit_generator(train_generator,
                                  epochs = 10,
                                  validation_data = test_generator,
                                  verbose=1)

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[100352,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradients/dense_1/MatMul_grad/MatMul_1 (defined at /home/nasserboan/miniconda3/envs/deep/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_14193]

Function call stack:
keras_scratch_graph


In [ ]:
import matplotlib.pyplot as plt

def plot_results(title,data,number) -> plt.figure:
    
    data = data.history
    f, ax = plt.subplots(1,3,figsize=(20,9))
    plt.suptitle(title,y=0.93,size=25)

    ax[0].plot(list(range(1,11)),data['loss'],color='tab:blue',label = 'train loss')
    ax[0].plot(list(range(1,11)),data['val_loss'],color='tab:orange',label = 'test loss')
    ax[0].grid(which='major',axis='x',alpha=0.2)
    ax[0].set_xticks(list(range(1,11)))
    ax[0].set_xticklabels(labels = list(range(1,11)),rotation=45)
    ax[0].legend()

    ax[1].plot(list(range(1,11)),data[f'auc_{number}'],color='tab:blue',label = 'train AUC')
    ax[1].plot(list(range(1,11)),data[f'val_auc_{number}'],color='tab:orange',label = 'test AUC')
    ax[1].grid(which='major',axis='x',alpha=0.2)
    ax[1].set_xticks(list(range(1,11)))
    ax[1].set_xticklabels(labels = list(range(1,11)),rotation=45)
    ax[1].legend()

    ax[2].plot(list(range(1,11)),data[f'false_negatives_{number}'],color='tab:blue',label = 'train false negatives')
    ax[2].plot(list(range(1,11)),data[f'val_false_negatives_{number}'],color='tab:orange',label = 'test false negatives')
    ax[2].grid(which='major',axis='x',alpha=0.2)
    ax[2].set_xticks(list(range(1,11)))
    ax[2].set_xticklabels(labels = list(range(1,11)),rotation=45)
    ax[2].legend()

#plt.savefig('lenet/lenet_train.png')